In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchmetrics.functional import mean_squared_error
import dgl
from dgl.data.rdf import AIFBDataset, MUTAGDataset, BGSDataset, AMDataset
from dgl.nn.pytorch import RelGraphConv
import argparse

In [3]:
torch.cuda.is_available()

False

# RGCN model

In [4]:
class RGCN(nn.Module):
    def __init__(self, num_nodes, h_dim, out_dim, num_rels):
        super().__init__()
        self.emb = nn.Embedding(num_nodes, h_dim)
        # two-layer RGCN
        self.conv1 = RelGraphConv(h_dim, h_dim, num_rels, regularizer='basis',
                                  num_bases=num_rels, self_loop=False)
        self.conv2 = RelGraphConv(h_dim, out_dim, num_rels, regularizer='basis',
                                  num_bases=num_rels, self_loop=False)

    def forward(self, g):
        x = self.emb.weight
        h = F.relu(self.conv1(g, x, g.edata[dgl.ETYPE], g.edata['norm']))
        h = self.conv2(g, h, g.edata[dgl.ETYPE], g.edata['norm'])
        return h

# Training

In [26]:
def evaluate(g, target_idx, labels, test_mask, model):
    test_idx = torch.nonzero(test_mask, as_tuple=False).squeeze()
    model.eval()
    with torch.no_grad():
        logits = model(g)
    logits = logits[target_idx]
    return mean_squared_error(logits[test_idx].squeeze(1), labels[test_idx])

def train(g, target_idx, labels, train_mask, model):
    # define train idx, loss function and optimizer
    train_idx = torch.nonzero(train_mask, as_tuple=False).squeeze()
#     loss_fcn = nn.CrossEntropyLoss()
    # Regression
    loss_fcn = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-2, weight_decay=5e-4)

    model.train()
    for epoch in range(50):
        logits = model(g)
        logits = logits[target_idx]
        loss = loss_fcn(logits[train_idx].squeeze(1), labels[train_idx].float())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        err = mean_squared_error(logits[train_idx].squeeze(1), labels[train_idx])
        print("Epoch {:05d} | Loss {:.4f} | Train MSE {:.4f} "
              . format(epoch, loss.item(), err))

# Load and Train

In [27]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Training with DGL built-in RGCN module.')

# load and preprocess dataset
data = AIFBDataset()

g = data[0]
g = g.int().to(device)
num_rels = len(g.canonical_etypes)
category = data.predict_category
labels = g.nodes[category].data.pop('labels')
train_mask = g.nodes[category].data.pop('train_mask')
test_mask = g.nodes[category].data.pop('test_mask')
# calculate normalization weight for each edge, and find target category and node id
for cetype in g.canonical_etypes:
    g.edges[cetype].data['norm'] = dgl.norm_by_dst(g, cetype).unsqueeze(1)
category_id = g.ntypes.index(category)
g = dgl.to_homogeneous(g, edata=['norm'])
node_ids = torch.arange(g.num_nodes()).to(device)
target_idx = node_ids[g.ndata[dgl.NTYPE] == category_id]
# create RGCN model    
in_size = g.num_nodes() # featureless with one-hot encoding
# out_size = data.num_classes
out_size = 1 # regression
model = RGCN(in_size, 16, out_size, num_rels).to(device)

train(g, target_idx, labels, train_mask, model)
mse_err = evaluate(g, target_idx, labels, test_mask, model)
print("Test accuracy {:.4f}".format(mse_err))

Training with DGL built-in RGCN module.
Done loading data from cached files.
Epoch 00000 | Loss 8.6259 | Train MSE 8.6259 
Epoch 00001 | Loss 89.5997 | Train MSE 89.5997 
Epoch 00002 | Loss 6.8727 | Train MSE 6.8727 
Epoch 00003 | Loss 6.1834 | Train MSE 6.1834 
Epoch 00004 | Loss 10.0802 | Train MSE 10.0802 
Epoch 00005 | Loss 6.2293 | Train MSE 6.2293 
Epoch 00006 | Loss 2.4950 | Train MSE 2.4950 
Epoch 00007 | Loss 1.0117 | Train MSE 1.0117 
Epoch 00008 | Loss 0.9046 | Train MSE 0.9046 
Epoch 00009 | Loss 1.2232 | Train MSE 1.2232 
Epoch 00010 | Loss 1.4605 | Train MSE 1.4605 
Epoch 00011 | Loss 1.4417 | Train MSE 1.4417 
Epoch 00012 | Loss 1.2370 | Train MSE 1.2370 
Epoch 00013 | Loss 0.9955 | Train MSE 0.9955 
Epoch 00014 | Loss 0.8586 | Train MSE 0.8586 
Epoch 00015 | Loss 0.7963 | Train MSE 0.7963 
Epoch 00016 | Loss 0.6840 | Train MSE 0.6840 
Epoch 00017 | Loss 0.5468 | Train MSE 0.5468 
Epoch 00018 | Loss 0.4538 | Train MSE 0.4538 
Epoch 00019 | Loss 0.4210 | Train MSE 0.4210 

# Explore the data

In [5]:
data = AIFBDataset()

Done loading data from cached files.


In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [14]:
g = data[0]
g = g.to(device)

In [24]:
g.nodes[category].data['train_mask']

tensor([0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0,
        1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1,
        0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0,
        1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=torch.uint8)

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Training with DGL built-in RGCN module.')

# load and preprocess dataset
data = AIFBDataset()

g = data[0]
g = g.int().to(device)
num_rels = len(g.canonical_etypes)
category = data.predict_category
labels = g.nodes[category].data.pop('labels')
train_mask = g.nodes[category].data.pop('train_mask')
test_mask = g.nodes[category].data.pop('test_mask')
# calculate normalization weight for each edge, and find target category and node id
for cetype in g.canonical_etypes:
    g.edges[cetype].data['norm'] = dgl.norm_by_dst(g, cetype).unsqueeze(1)
category_id = g.ntypes.index(category)
g = dgl.to_homogeneous(g, edata=['norm'])
node_ids = torch.arange(g.num_nodes()).to(device)
target_idx = node_ids[g.ndata[dgl.NTYPE] == category_id]
# create RGCN model    
in_size = g.num_nodes() # featureless with one-hot encoding
# out_size = data.num_classes
out_size = 1 # regression
model = RGCN(in_size, 16, out_size, num_rels).to(device)

train(g, target_idx, labels, train_mask, model)
mse_err = evaluate(g, target_idx, labels, test_mask, model)
print("Test accuracy {:.4f}".format(mse_err))

Training with DGL built-in RGCN module.
Done loading data from cached files.


/home/chenmin/miniconda3/envs/graph/lib/python3.9/site-packages/torch/nn/modules/loss.py:520: UserWarning: Using a target size (torch.Size([140])) that is different to the input size (torch.Size([140, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


RuntimeError: Found dtype Long but expected Float